# Why You (Likely) Do Not Need a Vector Database

First, some quick background. The large language models (LLMs) like GPT, Llama, etc all take a limited amount of context (16k tokens). As such if I want to use an LLM to perform question answering on a very large collection of documents, we cannot simply stuff all the documents into the context. While one could fine-tune the LLM on the set of documents, this is difficult and costly if the set of documents change very quickly. For instance, I have fantasized about running Llama on my laptop so I can easily find emails, documents, etc, and I certainly do want to finetune an LLM for every email I receive.

Retrieval Augmented Generation (RAG) has risen as a solution where you first try to find a small set of relevant documents that will help answer the question, and stuff just those documents into the context. A popular way to find these set of relevant documents is to compute a vector embedding of each document, which is a representation of the "meaning" of the document; documents which are similar semantically, should have vectors which are close. Then given a question, we  compute a vector embedding of the question and use nearest neighbor search to find the most relevant documents.

Vector databases have risen in popularity lately as a means of storing and computing nearest neighbor on a large collection of documents. However, I argue that you almost never need a vector database.

The task of finding a small set of documents that answers a given question is basically that of Information Retrieval, and very much predates vector databases. The most obvious forms of such systems which you interact with on a daily basis, are search engines (Google, Bing, Apache Lucene, Apple Spotlight, and many others). Of which highly scalable technologies such as reverse indexes are well known, highly available, and have been well developed over decades of research and engineering. Here, I will attempt to demonstrate why you probably do not need a vector database, and maybe some circumstances in which you might.

# Dataset
First, I need a benchmark dataset for document retrieval for the purposes of question answering. Here, we (mis)use the [SQuAD dataset](https://rajpurkar.github.io/SQuAD-explorer/).

The SQuAD dataset comprises of a collection of paragraphs, and questions for each paragraph. Each question is meant to answered only by information found in the paragraph. 
For instance a paragraph might be:
```
Beyoncé Giselle Knowles was born in Houston, Texas, to Celestine Ann "Tina" Knowles (née Beyincé), a hairdresser and salon owner, and Mathew Knowles, a Xerox sales manager. Beyoncé\'s name is a tribute to her mother\'s maiden name. Beyoncé\'s younger sister Solange is also a singer and a former member of Destiny\'s Child. Mathew is African-American, while Tina is of Louisiana Creole descent (with African, Native American, French, Cajun, and distant Irish and Spanish ancestry). Through her mother, Beyoncé is a descendant of Acadian leader Joseph Broussard. She was raised in a Methodist household.
```

And a few questions for this paragraph are:
 - What race was Beyonce's father?
 - Beyoncé was raised in what religion?
 - What is the name of Beyoncé's younger sister?
 
However, instead of answering the questions using the provided paragraph, we are going to *invert* the problem: given the question, find the paragraph containing the answer.

The dataset is not perfect for this purpose. For instance, there are questions such as "In what R&B group was she the lead singer?" which assume that there is sufficient context to disambiguate the pronoun "she". However, the *vast* majority of questions do not have this issue, and so suffices for our experiment here.

In [61]:
# We use a convenient little jupyter extension to memoize the result of costly / slow computations 
# (like openai calls etc). This memo is stored together with the repo. and makes things a lot easier to run.
# You do not have to selectively pickle and unpickle files and can always just run the notebook straight through.
%reload_ext xmemo_kernel_extension

In [22]:
import json
import pandas as pd
import pickle
import openai
import functools
import numpy as np
import concurrent.futures
import time

# Process SQuAD Dataset

In [3]:
d = pd.read_json('data/squad2.json')['data']
all_paragraphs = [paragraph['context'] for ent in d for paragraph in ent['paragraphs']]

In [4]:
# Ok. list comprehension is still possible but it gets a little obnoxious.
all_qa = []
paragraph_id = 0
for ent in d:
    for paragraph in ent['paragraphs']:
        for qa in paragraph['qas']:
            if len(qa['answers']) > 0 and qa['is_impossible'] == False:
                all_qa.append((paragraph_id, qa['question'], qa['answers'][0]['text']))
        paragraph_id += 1

In [5]:
len(all_qa)

86821

In [6]:
len(all_paragraphs)

19035

### There are ~ 87k questions and 19k paragraphs.

### Some paragraph samples:

In [7]:
print(all_paragraphs[3])
print(all_paragraphs[10])

Beyoncé Giselle Knowles was born in Houston, Texas, to Celestine Ann "Tina" Knowles (née Beyincé), a hairdresser and salon owner, and Mathew Knowles, a Xerox sales manager. Beyoncé's name is a tribute to her mother's maiden name. Beyoncé's younger sister Solange is also a singer and a former member of Destiny's Child. Mathew is African-American, while Tina is of Louisiana Creole descent (with African, Native American, French, Cajun, and distant Irish and Spanish ancestry). Through her mother, Beyoncé is a descendant of Acadian leader Joseph Broussard. She was raised in a Methodist household.
Beyoncé's first solo recording was a feature on Jay Z's "'03 Bonnie & Clyde" that was released in October 2002, peaking at number four on the U.S. Billboard Hot 100 chart. Her first solo album Dangerously in Love was released on June 24, 2003, after Michelle Williams and Kelly Rowland had released their solo efforts. The album sold 317,000 copies in its first week, debuted atop the Billboard 200, a

### Some question samples:

In [8]:
print(all_qa[40:42])
print(all_qa[130:132])

[(3, 'Where did Beyonce get her name from?', "her mother's maiden name"), (3, "What race was Beyonce's father?", 'African-American')]
[(10, "Beyonce's first solo album in the U.S. with what artist in the lead single?", 'Jay Z'), (10, 'What solo album did Beyonce release in 2003?', 'Dangerously in Love')]


# Open AI RAG QA Example
Just to give a quick example of how RAG might be used on this dataset, we build a simple question answering agent which answers a given quesion using a context.

In [9]:
client = openai.OpenAI(max_retries=5,timeout=10)

In [10]:
def question_answerer(client, question, context):
    system_prompt = "You are an assistant for question-answering tasks. Use the provided pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Provide just the answer in as few words as possible. Do not use complete sentences."
    user_prompt = f"Question: {question} \nContext: {context} \nAnswer:"
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt,
            }
        ],
        model="gpt-3.5-turbo",
        temperature=0.0,
    )
    response = response.choices[0].message.content
    return response

This is the first question answer pair we have in the dataset:

In [11]:
para_id, question, answer = all_qa[0]
print("Context: ", all_paragraphs[para_id])
print("Question: ", question)
print("Correct Answer: ": answer)

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
When did Beyonce start becoming popular?
in the late 1990s


And here is the response from OpenAI. It seems to work pretty well!

In [12]:
response = question_answerer(client, question, all_paragraphs[para_id])
print(response)

Late 1990s


However, how do we tell that "in the late 1990s" is the same answer and "Late 1990s"? There are some heuristics like BLEU which can be used. However,a lot of careful normalization is needed. For instance how do we deal with different ways of expressing numbers such as "10" vs "ten"? "1990s" vs "90s"? "1,200" vs "1200"? Also the answers in the dataset are  pretty short, so a threshold can be quite difficult to define. 

So how do we evaluate? Use an LLM to evaluate the answers too!

## Using an LLM to compare answers

It has been shown in many domains that an LLMs can actually match human performance in evaluating answers. And the intuition behind this is that it is easier to evaluate than it create. So here we use an LLM to compare two answers to a question. We are simply asking the LLM if the answers mean the same thing. And it seems to work quite nicely for our purposes. Of course, for a real application, we would actually like to evaluate this evaluator against a real dataset, but we are just doing a quick and dirty experiment here. So this will do.

In [13]:
def is_same(client, question, a1, a2):
    system_prompt = "You are an assistant for scoring answers. Two answers to a hypothetical question are provided. Say 'Yes' if both answers have the same meaning, and 'No' otherwise."
    user_prompt = f"Question: {question} \Answer 1: {a1} \nAnswer 2: {a2}"
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt,
            }
        ],
        model="gpt-3.5-turbo",
        temperature=0.0
    )
    response = response.choices[0].message.content == 'Yes'
    return response
    

In [14]:
answer

'in the late 1990s'

In [15]:
is_same(client, question, answer, "Late 1990s")

True

In [16]:
is_same(client, question, answer, "Late 1980s")

False

# Evaluate RAG
We can now evalute the performance our RAG set up!. Just to keep things cheap and fast, we just run a 1% subset and take a look at the results!

In [29]:
import numpy as np
sub_qa = all_qa[0:len(all_qa):100]

In [18]:
import concurrent.futures

In [19]:
def do_work(client, question, true_answer, context):
    response = question_answerer(client, question, context)
    evaluation = is_same(client, question, true_answer, response)
    return true_answer, response, evaluation

In [30]:
%%xmemo input=sub_qa output=responses,evaluations
responses = []
evaluations = []
ctr = 0
for ent in sub_qa:
    para_id, question, true_answer = ent
    try:
        true_answer, response, evaluation = do_work(client, question, true_answer, all_paragraphs[para_id])
    except openai.RateLimitError:
        time.sleep(10)
        true_answer, response, evaluation = do_work(client, question, true_answer, all_paragraphs[para_id])
    ctr += 1
    print(f"Correct Answer: {true_answer}, Response: {response}, Eval: {evaluation}. {ctr}/{len(sub_qa)}")
    responses.append(response)
    evaluations.append(evaluation)

Correct Answer: in the late 1990s, Response: Late 1990s, Eval: True. 1/869
Correct Answer: eleven, Response: 11 weeks, Eval: True. 2/869
Correct Answer: ten, Response: Ten, Eval: True. 3/869
Correct Answer: Beck, Response: Beck, Eval: True. 4/869
Correct Answer: Forbes, Response: Forbes, Eval: True. 5/869
Correct Answer: Jarett Wieselman, Response: Jarett Wieselman., Eval: True. 6/869
Correct Answer: 8 million, Response: 8 million, Eval: True. 7/869
Correct Answer: in Destiny's Child's shows and tours, Response: Destiny's Child's shows and tours., Eval: True. 8/869
Correct Answer: Polish, Response: Polish., Eval: True. 9/869
Correct Answer: Rondo Op. 1., Response: Rondo Op. 1, Eval: True. 10/869
Correct Answer: Polish, Response: Polish, Eval: True. 11/869
Correct Answer: Pleyel, Response: Pleyel, Eval: True. 12/869
Correct Answer: 1830, Response: 1830, Eval: True. 13/869
Correct Answer: Clésinger, Response: Clésinger, Eval: True. 14/869
Correct Answer: Karol Szymanowski, Response: Karo

Correct Answer: the Royal George, Response: Royal George, Eval: False. 111/869
Correct Answer: The Angel, Response: The Angel, Eval: True. 112/869
Correct Answer: multiple ISPs interconnect at peering points or Internet exchange points, Response: Interconnection between ISPs., Eval: False. 113/869
Correct Answer: Manga! Manga! The World of Japanese Comics, Response: Manga! Manga! The World of Japanese Comics, Eval: True. 114/869
Correct Answer: Royal, Response: Royal Navy, Eval: True. 115/869
Correct Answer: emigration, Response: Emigration, Eval: True. 116/869
Correct Answer: Christmas Day 1967, Response: Christmas Day 1967, Eval: True. 117/869
Correct Answer: Preaspirated stops, Response: preaspirated stops, Eval: True. 118/869
Correct Answer: link two metal centers, Response: Metal centers, Eval: False. 119/869
Correct Answer: as a tracer gas for minute leak detection, Response: Leak detection., Eval: False. 120/869
Correct Answer: Project Mercury, Response: Project Mercury, Eval: T

Correct Answer: handbook for witch-hunters, Response: A handbook for witch-hunters., Eval: True. 212/869
Correct Answer: 1991, Response: 1991, Eval: True. 213/869
Correct Answer: 1927, Response: 1927, Eval: True. 214/869
Correct Answer: a control system or central controller), Response: Control system, central controller, Eval: True. 215/869
Correct Answer: umm walad, Response: Umm walad, Eval: True. 216/869
Correct Answer: Etymologies, Response: Etymologies, Eval: True. 217/869
Correct Answer: Clinton used his knowledge of black culture to exploit black people for political gain, Response: Because they believed Clinton exploited black culture for political gain and did not serve black interests., Eval: True. 218/869
Correct Answer: 19th century, Response: 19th century, Eval: True. 219/869
Correct Answer: 87%, Response: 87%, Eval: True. 220/869
Correct Answer: 1931, Response: 1931, Eval: True. 221/869
Correct Answer: 35, Response: 35, Eval: True. 222/869
Correct Answer: 2011, Response:

Correct Answer: police and public works departments, Response: police and public works departments, Eval: True. 309/869
Correct Answer: take action on Ethiopia, Response: Take action on Ethiopia., Eval: True. 310/869
Correct Answer: antimony, Response: Antimony, Eval: True. 311/869
Correct Answer: hexavalent, Response: hexavalent uranium compounds, Eval: True. 312/869
Correct Answer: 845, Response: 845, Eval: True. 313/869
Correct Answer: glucose metabolism, Response: Altered glucose metabolism., Eval: True. 314/869
Correct Answer: governess, Response: Governess, Eval: True. 315/869
Correct Answer: Windlesham Moor, Response: Windlesham Moor, Eval: True. 316/869
Correct Answer: ambiguity regarding the definition of sexual orientation, Response: Ambiguity, difficulty determining., Eval: False. 317/869
Correct Answer: both the masculine and the feminine sides of their natures, Response: Desires derived from both masculine and feminine sides., Eval: True. 318/869
Correct Answer: Recognizin

Correct Answer: the Roman Catholic Church and local rulers, Response: Roman Catholic Church and local rulers, Eval: True. 406/869
Correct Answer: fourth largest, Response: Fourth, Eval: True. 407/869
Correct Answer: The whale shark, Response: Whale shark, Eval: True. 408/869
Correct Answer: July 18, 1896, Response: July 18, 1896., Eval: True. 409/869
Correct Answer: Spencer W. Kimball Tower, Response: Spencer W. Kimball Tower, Eval: True. 410/869
Correct Answer: 1784, Response: 1784, Eval: True. 411/869
Correct Answer: 1877, Response: 1877, Eval: True. 412/869
Correct Answer: trade secrets, Response: Trade secret, Eval: True. 413/869
Correct Answer: the Seminoles lands west of the Mississippi River if they agreed to leave Florida. Many Seminole left at this time., Response: Lands west of the Mississippi River, Eval: False. 414/869
Correct Answer: Humpy Bong, Response: Queen, Eval: False. 415/869
Correct Answer: Bob Geldof and Midge Ure, Response: Bob Geldof and Midge Ure, Eval: True. 4

Correct Answer: Stop TB Partnership, Response: Stop TB Partnership, Eval: True. 499/869
Correct Answer: Executive Order 8802, Response: Executive Order 8802, Eval: True. 500/869
Correct Answer: an effort towards inclusion rather than a discriminatory practice, Response: Effort towards inclusion., Eval: True. 501/869
Correct Answer: more than 60, Response: More than 60., Eval: True. 502/869
Correct Answer: as soon as the final has finished, in order to be ready in time for the presentation ceremony., Response: After the final., Eval: False. 503/869
Correct Answer: Governor's Foot Guard, Response: Governor's Foot Guard, Eval: True. 504/869
Correct Answer: Not all of these small streams, Response: No., Eval: True. 505/869
Correct Answer: Bobby Seale, Response: Bobby Seale, Eval: True. 506/869
Correct Answer: The Beinecke Rare Book and Manuscript, Response: Beinecke Rare Book and Manuscript Library, Eval: True. 507/869
Correct Answer: Grove Street Cemetery, Response: Grove Street Cemetery,

Correct Answer: LaserDiscs, Response: LaserDiscs use analog video., Eval: True. 592/869
Correct Answer: David Bowie, Response: David Bowie, Eval: True. 593/869
Correct Answer: division of powers between two levels of government of equal status, Response: Yes., Eval: True. 594/869
Correct Answer: federal structure, Response: Consent, Eval: False. 595/869
Correct Answer: arthropods, Response: Arthropods, Eval: True. 596/869
Correct Answer: deuterostomes, Response: Deuterostomes, Eval: True. 597/869
Correct Answer: "a universe with a god would be a completely different kind of universe from one without, and it would be a scientific difference.", Response: Empirical question., Eval: False. 598/869
Correct Answer: 12, Response: 12, Eval: True. 599/869
Correct Answer: November 2002, Response: November 2002, Eval: True. 600/869
Correct Answer: deeply unpopular, Response: Unpopular., Eval: True. 601/869
Correct Answer: the early 1960s, Response: early 1960s, Eval: True. 602/869
Correct Answer:

Correct Answer: Allen Daviau, Response: Allen Daviau, Eval: True. 687/869
Correct Answer: the controller, Response: Traction machine., Eval: False. 688/869
Correct Answer: underground public elevator, Response: 3-station underground public elevator, Eval: False. 689/869
Correct Answer: hydrogen and helium, Response: Hydrogen and helium., Eval: True. 690/869
Correct Answer: southern cyclonic storm, Response: The Scooter is a storm., Eval: False. 691/869
Correct Answer: overhead wires, Response: Both third rail and overhead wires are used., Eval: False. 692/869
Correct Answer: electric, Response: Electric trains, Eval: True. 693/869
Correct Answer: Spanish is the most widely taught language in American colleges and universities with 53 percent of the total number of people enrolled, Response: Spanish language classes are popular in the U.S., Eval: True. 694/869
Correct Answer: 1970, Response: 1970, Eval: True. 695/869
Correct Answer: 300, Response: 300, Eval: True. 696/869
Correct Answer

Correct Answer: display constantly updated information and content, Response: Display constantly updated information., Eval: False. 771/869
Correct Answer: Common Market for Eastern and Southern Africa, Response: Common Market for Eastern and Southern Africa, Eval: True. 772/869
Correct Answer: Botswana, Lesotho and Swaziland, Response: Botswana, Lesotho, Swaziland, Eval: True. 773/869
Correct Answer: on-demand human-translation services, Response: On-demand human translation., Eval: True. 774/869
Correct Answer: slope-line approach system, Response: Slope-line approach system., Eval: True. 775/869
Correct Answer: 10th century, Response: 10th century, Eval: True. 776/869
Correct Answer: US$199, Response: $199, Eval: True. 777/869
Correct Answer: marshland, Response: Eastern Arabia, Eval: False. 778/869
Correct Answer: outside the city walls in graveyards, Response: Outside the city walls in graveyards., Eval: True. 779/869
Correct Answer: Salai, Response: Salai, Eval: True. 780/869
Cor

Correct Answer: Public Broadcasting, Response: Public Broadcasting Service (PBS), Eval: True. 861/869
Correct Answer: growth, Response: Growth medium., Eval: True. 862/869
Correct Answer: poaching, Response: Poaching., Eval: True. 863/869
Correct Answer: pests, Response: pests, Eval: True. 864/869
Correct Answer: 5,200,000 acres, Response: National Wildlife Refuge System lands., Eval: True. 865/869
Correct Answer: Migratory Bird Hunting Stamp Act, Response: Migratory Bird Hunting Stamp Act, Eval: True. 866/869
Correct Answer: Bagmati, Response: Bagmati, Eval: True. 867/869
Correct Answer: northwestern, Response: Northwestern part, Eval: True. 868/869
Correct Answer: National Academy of Medical Sciences, Response: National Academy of Medical Sciences, Eval: True. 869/869


In [31]:
%%xmemo input=sub_qa,responses,evaluations output=df
df = pd.DataFrame()
df['context'] = [all_paragraphs[i[0]] for i in sub_qa]
df['question'] = [i[1] for i in sub_qa]
df['answer'] = [i[2] for i in sub_qa]
df['response'] = responses
df['evaluations'] = evaluations

In [32]:
df

,context,question,answer,response,evaluations
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,Late 1990s,True
1,"The remaining band members recorded ""Independe...","How many weeks did their single ""Independent W...",eleven,11 weeks,True
2,"At the 52nd Annual Grammy Awards, Beyoncé rece...",How many awards was Beyonce nominated for at t...,ten,Ten,True
3,At the 57th Annual Grammy Awards in February 2...,Which artist beat Beyonce out for Album of the...,Beck,Beck,True
4,Forbes magazine began reporting on Beyoncé's e...,In 2012 who placed Beyonce at 16 in the Celebr...,Forbes,Forbes,True
...,...,...,...,...,...
864,"Each year, nearly $200 million in hunters' fed...",What does land has Federal Duck Stamp money he...,"5,200,000 acres",National Wildlife Refuge System lands.,True
865,"On 16 March 1934, President Franklin D. Roosev...",What act was signed in 1934?,Migratory Bird Hunting Stamp Act,Migratory Bird Hunting Stamp Act,True
866,Kathmandu is located in the northwestern part ...,What river is south of Kathmandu?,Bagmati,Bagmati,True
867,Swayambhu is a Buddhist stupa atop a hillock a...,In what part of Kathmandu is Swayambhu located?,northwestern,Northwestern part,True


# OpenAI Embeddings

In [51]:
ctr = 0
def embed_entry(text):
    global ctr
    while True:
        try:
            response = client.embeddings.create(input=text, model='text-embedding-ada-002', encoding_format='float')
            ctr += 1
            if ctr % 100 == 0:
                print(ctr)
        except openai.RateLimitError as e:
            time.sleep(1)
            continue
        break
    return response.data[0].embedding

def generate_openai_embeddings(client, textlist):
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        ret = list(executor.map(embed_entry, textlist))
    return ret

In [62]:
%%xmemo input=embed_entry,all_paragraphs output=par_embeddings
par_embeddings = generate_openai_embeddings(client, all_paragraphs)
par_embeddings = np.array(par_embeddings)

Loading from 6f9fdedb188cbc14a74921efe44e51202f2878fd483cc3648fa869888fd39322.pickle


In [63]:
%%xmemo input=embed_entry,all_qa output=qn_embeddings
qn_embeddings = generate_openai_embeddings(client, [q[1] for q in all_qa])
qn_embeddings = np.array(qn_embeddings)

800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
19

# Nearest Neighbor

In [2]:
from sklearn.neighbors import NearestNeighbors

In [3]:
par_embeddings_arr = np.load('data/openai_paragraphs_embeddings.npy')
qn_embeddings_arr = np.load('data/openai_qn_embeddings.npy')

In [4]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
nbrs.fit(par_embeddings_arr)

NearestNeighbors(algorithm='ball_tree', n_neighbors=10)

In [7]:
knn = nbrs.kneighbors(qn_embeddings_arr)

In [8]:
knn

(array([[0.53342373, 0.54183945, 0.5480534 , ..., 0.55267631, 0.55477935,
         0.55652812],
        [0.51625546, 0.56791918, 0.57577417, ..., 0.60289394, 0.60317292,
         0.60357273],
        [0.47841394, 0.49049756, 0.51447069, ..., 0.54367424, 0.5441335 ,
         0.54597068],
        ...,
        [0.57681319, 0.59904399, 0.59999977, ..., 0.6253042 , 0.62548659,
         0.62892828],
        [0.49743956, 0.57099157, 0.5722643 , ..., 0.59779   , 0.60029111,
         0.60105112],
        [0.6205644 , 0.67376102, 0.68988245, ..., 0.70489665, 0.70647834,
         0.70827299]]),
 array([[   23,    45,    26, ...,    50,     0,    14],
        [    4,     5,     3, ...,    64,    42,    20],
        [    1,    11,     8, ...,     7,    23,    36],
        ...,
        [18979, 18929, 18925, ..., 18966, 18972, 18934],
        [18979, 18932, 18940, ..., 18978, 18972, 18941],
        [18979, 18255,  3350, ...,   126, 13786, 11440]]))

In [9]:
np.save('data/openai_knn_qn_to_paragraph.npy', knn)

In [10]:
_, nearest_paragraphs = knn

In [11]:
nearest_paragraphs.shape

(86821, 10)

In [14]:
truth = np.array([i[0] for i in all_qa])

In [30]:
p_at = []
p_at.append((nearest_paragraphs[:,0] == truth).sum())
for i in range(1,10):
    p = (nearest_paragraphs[:,i] == truth).sum()
    p_at.append(p_at[i-1] + p)

In [31]:
p_at

[54566, 63738, 67928, 70421, 72127, 73358, 74400, 75232, 75857, 76395]

# Simpler Embeddings

In [34]:
from rank_bm25 import BM25Okapi

In [119]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/ylow/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [135]:
import nltk.tokenize
import concurrent

In [130]:
def tokenize(sentence):
    words = nltk.tokenize.word_tokenize(sentence)
    return [word.lower() for word in words if word.isalpha()]

In [132]:
tokenized_corpus = [tokenize(par) for par in all_paragraphs]
bm25 = BM25Okapi(tokenized_corpus)

In [21]:
import importlib
import bm25_helpers
importlib.reload(bm25_helpers)

<module 'bm25_helpers' from '/Users/ylow/xetrepos/RagIRBench/bm25_helpers.py'>

In [160]:
bm25_nearest_paragraphs = []
futures = []
ctr = 0
nworkers = 8
with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
    for s in range(nworkers):
        start = (s * len(all_qa)) // nworkers
        end = ((s + 1) * len(all_qa)) // nworkers
        if s == nworkers - 1:
            end = len(all_qa)
        futures.append(executor.submit(bm25_helpers.get_bm25_top_10_batch, bm25, [qa[1] for qa in all_qa[start:end]]))

    for future in futures:
        bm25_nearest_paragraphs.extend(future.result())
        print(len(bm25_nearest_paragraphs))


5426
10852
16278
21705
27131
32557
37984
43410
48836
54263
59689
65115
70542
75968
81394
86821


In [161]:
bm25_nearest_paragraphs_arr = np.array(bm25_nearest_paragraphs)

In [72]:
np.save('data/bm25_knn_qn_to_paragraph.npy', bm25_nearest_paragraphs_arr)

In [37]:
bm25_nearest_paragraphs_arr = np.load('data/bm25_knn_qn_to_paragraph.npy')

In [38]:
bm25_nearest_paragraphs_arr

array([[ 6025, 15710, 10808, ..., 11323,  5977, 17616],
       [ 9402,  9371, 10274, ..., 11248, 17503,  1679],
       [   36,     8,     1, ...,     7,  3685,    21],
       ...,
       [18938, 18935, 18936, ..., 18945, 18933, 18963],
       [18939, 18945, 18940, ..., 18976, 18932, 18934],
       [14150, 14946,  1576, ..., 13842,  9371, 10836]])

In [39]:
bm25_p_at = []
bm25_p_at.append((bm25_nearest_paragraphs_arr[:,0] == truth).sum())
for i in range(1,10):
    p = (bm25_nearest_paragraphs_arr[:,i] == truth).sum()
    bm25_p_at.append(bm25_p_at[i-1] + p)

In [40]:
bm25_p_at

[42140, 49077, 52628, 54824, 56524, 57858, 58863, 59752, 60493, 61126]

# Query Augmentation

In [3]:
query_augment_cache = {}
try:
    query_augment_cache = pickle.load(open('data/query_augment_cache.pickle','rb'))
except:
    pass

In [4]:
def query_augment(client, question):
    global query_augment_cache
    key = question
    if key in query_augment_cache:
        return query_augment_cache[key]

    system_prompt = "You are to perform query augmentation for a question answering system. For the provided question, return a comma separated list of up to 5 new words which are not part of the question that can be used to improve document retrieval. These words will be added to the words already in the question and will be used to search a collection of documents to find a document which may contain the answer."
    user_prompt = f"Question: {question}"
    message = [
                {
                    "role": "system",
                    "content": system_prompt,
                },
                {
                    "role": "user",
                    "content": user_prompt,
                }]
    while True:
        try:
            response = client.chat.completions.create(
                    messages=message,
                    model="gpt-3.5-turbo",
                    temperature=0.0)
        except openai.RateLimitError as e:
            print(e)
            time.sleep(1)
            continue
        break

    # split by , 
    r = response.choices[0].message.content
    response = response.choices[0].message.content.split(',')
    # split by space
    response = [r.strip().split(' ') for r in response]
    # flatten
    response = [word for i in response for word in i]
    response = list(set(response))
    query_augment_cache[key] = response
    return response


In [5]:
client = openai.OpenAI(max_retries=5,timeout=30)

In [ ]:
import time
import concurrent
question_augmentation = []
def augment_lambda(ent):
    ret = query_augment(client, ent[1])
    print(ret)
    return ret
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    question_augmentation = list(executor.map(augment_lambda, all_qa))

In [7]:
f = open('data/query_augment_cache.pickle','wb')
pickle.dump(query_augment_cache, f)
f.close()

In [20]:
f = open('data/qa_question_augmentation.pickle','wb')
pickle.dump(question_augmentation, f)
f.close()

# Using Query Augmentation

In [23]:
from rank_bm25 import BM25Okapi
tokenized_corpus = [bm25_helpers.tokenize(par) for par in all_paragraphs]
bm25 = BM25Okapi(tokenized_corpus)

In [25]:
bm25_nearest_paragraphs = []
futures = []
ctr = 0
nworkers = 8
with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
    for s in range(nworkers):
        start = (s * len(all_qa)) // nworkers
        end = ((s + 1) * len(all_qa)) // nworkers
        if s == nworkers - 1:
            end = len(all_qa)
        futures.append(executor.submit(bm25_helpers.get_bm25_top_10_batch_augmented, bm25, 
                                       [qa[1] for qa in all_qa[start:end]],
                                       question_augmentation[start:end]))

    for future in futures:
        bm25_nearest_paragraphs.extend(future.result())
        print(len(bm25_nearest_paragraphs))

10852
21705
32557
43410
54263
65115
75968
86821


In [27]:
bm25_nearest_paragraphs_arr = np.array(bm25_nearest_paragraphs)

In [33]:
np.save('data/bm25_knn_qn_to_paragraph_augmented.npy', bm25_nearest_paragraphs_arr)

In [29]:
truth = np.array([i[0] for i in all_qa])

In [31]:
bm25_p_at = []
bm25_p_at.append((bm25_nearest_paragraphs_arr[:,0] == truth).sum())
for i in range(1,10):
    p = (bm25_nearest_paragraphs_arr[:,i] == truth).sum()
    bm25_p_at.append(bm25_p_at[i-1] + p)

In [32]:
bm25_p_at

[50821, 59938, 64156, 66789, 68640, 70104, 71252, 72189, 73027, 73692]